In [ ]:
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.pipeline.engine import Workflow, Node

from nipype.interfaces.freesurfer import Binarize, MRIConvert, FSCommand
from nipype.interfaces.fsl import ApplyMask, Reorient2Std
from nipype.interfaces.fsl.preprocess import MCFLIRT, SliceTimer, FLIRT, FAST
from nipype.interfaces.spm.preprocess import Smooth
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.interfaces.fsl.model import GLM
from nipype.algorithms.confounds import CompCor
from nipype.interfaces.nipy.preprocess import Trim

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

#other study-specific variables
project_home = '/Users/catcamacho/Dropbox/Projects/TH_NAR_ASL/proc'
raw_dir = project_home + '/raw'
subjects_list = open(project_home + '/misc/subjects.txt').read().splitlines()
output_dir = project_home + '/proc/preprocessing'
wkflow_dir = project_home + '/workflows'
template = project_home + '/template/MNI152_T1_2mm_brain.nii'

#freesurfer setup
subjects_dir = project_home + '/freesurfer'
FSCommand.set_default_subjects_dir(subjects_dir)

#Population specific variables for ASL
nex_asl = 3 #number of excitations from the 3D ASL scan parameters
inversion_efficiency = 0.8 #from GE
background_supp_eff = 0.75 #from GE
efficiency = inversion_efficiency * background_supp_eff 
T1_blood = 1.6 #T1 of blood in seconds(1.6s at 3T and 1.4s at 1.5T)
sat_time = 2 #in seconds, from GE
partition_coeff = 0.9 #whole brain average in ml/g
scaling_factor = 32 #scaling factor, can be taken from PW dicom header at position 0043,107f (corresponds to #coils?)
postlabel_delay = 1.525 #post label delay in seconds
labeling_time = 1.450 #labeling time in seconds
T1_tissue = 1.2 #estimated T1 of grey matter in seconds
TR = 4.844 #repetition time


In [ ]:
## File handling nodes

# Select subjects
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# SelectFiles
templates = {'pw_volume': raw_dir + '/{subjid}/pw.nii',
             'rest': raw_dir + '/{subjid}/rest_raw.nii',
             'pd_volume': raw_dir + '/{subjid}/pd.nii'}
selectfiles = Node(SelectFiles(templates), name='selectfiles')

# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(FreeSurferSource(subjects_dir=subjects_dir),
                run_without_submitting=True,
                name='fssource')
# Datasink
datasink = Node(DataSink(base_directory = output_dir, 
                         container = output_dir), 
                name='datasink')

# DataSink output substitutions (for ease of folder naming)
substitutions = [('_subjid_', '')]
datasink.inputs.substitutions = substitutions

# Grab processed anat
struct_template = {'proc_anat':output_dir + '/preproc_anat/{subjid}/brainmask_out_reoriented.nii'}
structgrabber = Node(SelectFiles(struct_template),name='structgrabber')

In [ ]:
## File Processing nodes

# convert files to nifti
mri_convert = Node(MRIConvert(out_type='nii',
                              conform_size=2,
                              crop_size= (128, 128, 128)), 
                   name='mri_convert')

# reorient data for consistency
reorient_anat = Node(Reorient2Std(output_type='NIFTI'),
                     name='reorient_anat')

# Binarize -  binarize and dilate image to create a brainmask
binarize = Node(Binarize(min=0.5,
                         max=300,
                         dilate=3,
                         erode=2,
                         out_type='nii'),
                name='binarize')

# reorient PD/PW data for consistency
reorient_pd = Node(Reorient2Std(output_type='NIFTI'),
                   name='reorient_pd')

reorient_pw = Node(Reorient2Std(output_type='NIFTI'),
                   name='reorient_pw')

# Register subject's PD/PW to subject anatomy
reg_pw2anat = Node(FLIRT(output_type='NIFTI',
                         out_file='warped_pw.nii'),
                   name='reg_pw2anat')

reg_pd2anat = Node(FLIRT(output_type='NIFTI',
                         apply_xfm = True, 
                         out_file='warped_pd.nii'),
                   name='reg_pd2anat')

# Mask brain in pw volume
applyMask_pw = Node(ApplyMask(output_type='NIFTI'),
                    name='applyMask_pw')

# Mask brain pd volumes
applyMask_pd = Node(ApplyMask(output_type='NIFTI'), 
                    name='applyMask_pd')

In [ ]:
## Custom functions
#quantify CBF from PW volume (Alsop MRIM 2015 method)
def quantify_cbf_alsop(pw_volume,pd_volume,sat_time,postlabel_delay,T1_blood,labeling_time,efficiency,partition_coeff,TR,T1_tissue,scaling_factor,nex_asl):
    import os
    import nibabel as nib
    from numpy import exp
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    # set variables
    pw_nifti1 = nib.load(pw_volume)
    pw_data = pw_nifti1.get_data()
    pw_data = pw_data.astype(float)
    pd_nifti1 = nib.load(pd_volume)
    pd_data = pd_nifti1.get_data()
    pd_data = pd_data.astype(float)
    conversion = 6000 #to convert values from mL/g/s to mL/100g/min
    pd_factor = 1/(1-exp((-1*TR)/T1_tissue))
    
    cbf_numerator = conversion*partition_coeff*pw_data*exp(postlabel_delay/T1_blood)
    cbf_denominator = sat_time*efficiency*T1_blood*scaling_factor*nex_asl*pd_data*pd_factor*(1-exp((-1*labeling_time)/T1_blood))
    cbf_data = cbf_numerator/cbf_denominator
    
    cbf_volume = nib.Nifti1Image(cbf_data, pw_nifti1.affine)
    nib.save(cbf_volume, 'alsop_cbf.nii')
    cbf_path = os.path.abspath('alsop_cbf.nii')
    return cbf_path
    
def make_histogram(cbf_vol):
    from os.path import abspath
    from nibabel import load
    from numpy import isnan
    from matplotlib import pyplot as plt
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    cbf_image = load(cbf_vol)
    cbf_data = cbf_image.get_data()

    plt.Figure()
    data = cbf_data.flatten()
    plt.hist(data[~isnan(data)], bins = range(0,200,10))
    plt.title("Distribution of CBF values in mL/100g/min")
    plt.savefig('cbf_histogram.png')
    cbf_hist = abspath('cbf_histogram.png')
    return(cbf_hist)

def create_cbfcoreg_plot(ref,anat):
    from os.path import abspath
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    
    coreg_filename='coregistration.png'
    display = plotting.plot_anat(ref, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'coregistration to anatomy')
    display.add_edges(anat)
    display.savefig(coreg_filename) 
    display.close()
    coreg_file = abspath(coreg_filename)
    
    return(coreg_file)

In [ ]:
## Normalizing data for first and second level analysis

quant_cbf_alsop = Node(Function(input_names=['pw_volume','pd_volume',
                                             'sat_time','postlabel_delay',
                                             'T1_blood','labeling_time',
                                             'efficiency','partition_coeff',
                                             'TR','T1_tissue','scaling_factor',
                                             'nex_asl'],
                                output_names=['cbf_volume'],
                                function=quantify_cbf_alsop),
                       name='quant_cbf_alsop')
quant_cbf_alsop.inputs.sat_time=sat_time
quant_cbf_alsop.inputs.postlabel_delay=postlabel_delay
quant_cbf_alsop.inputs.T1_blood=T1_blood
quant_cbf_alsop.inputs.labeling_time=labeling_time
quant_cbf_alsop.inputs.efficiency=efficiency
quant_cbf_alsop.inputs.partition_coeff=partition_coeff
quant_cbf_alsop.inputs.TR=TR
quant_cbf_alsop.inputs.T1_tissue=T1_tissue
quant_cbf_alsop.inputs.scaling_factor=scaling_factor
quant_cbf_alsop.inputs.nex_asl=nex_asl

# make histgram of CBF values for each subject
makecbf_hist = Node(Function(input_names=['cbf_vol'], 
                             output_names=['cbf_hist'], 
                             function=make_histogram), 
                    name = 'makecbf_hist')

checkcoreg_cbf = Node(Function(input_names=['ref','anat'],
                               output_names=['coreg_file'],
                               function=create_cbfcoreg_plot), 
                      name='checkcoreg_cbf')

checkcoreg_mni = Node(Function(input_names=['ref','anat'],
                               output_names=['coreg_file'],
                               function=create_cbfcoreg_plot), 
                      name='checkcoreg_mni')
checkcoreg_mni.inputs.anat=template

# Register subject's anatomy/CBF to the template
reg_anat2mni = Node(FLIRT(output_type='NIFTI',
                          reference=template),
                    name='reg_anat2mni')

reg_cbf2mni = Node(FLIRT(output_type='NIFTI',
                         apply_xfm = True,
                         reference=template,
                         out_file='warped_cbf.nii'),
                   name='reg_cbf2mni')

# smooth cbf data
cbf_smooth = Node(Smooth(fwhm=[6,6,6], 
                     implicit_masking=True), 
              name='smooth')

In [ ]:
# Create a flow for preprocessing anat + asl volumes 
cbfpreproc = Workflow(name='cbfpreproc')

# Connect all components of the preprocessing workflow
cbfpreproc.connect([(infosource,selectfiles, [('subjid', 'subjid')]),
                    (infosource,fssource, [('subjid','subject_id')]),
                    (fssource,mri_convert, [('brainmask', 'in_file')]),
                    (mri_convert,reorient_anat, [('out_file','in_file')]),
                    (reorient_anat,binarize, [('out_file','in_file')]),
                    (reorient_anat,reg_pw2anat, [('out_file','reference')]),
                    (reorient_anat,reg_pd2anat, [('out_file','reference')]),
                    (reorient_anat,checkcoreg_cbf, [('out_file','anat')]),
                    (reorient_anat,reg_anat2mni, [('out_file','in_file')]),
                    (reg_anat2mni,reg_cbf2mni, [('out_matrix_file','in_matrix_file')]),
                    (reg_anat2mni,checkcoreg_mni, [('out_file','ref')]),
                    (binarize,applyMask_pw, [('binary_file','mask_file')]),
                    (binarize,applyMask_pd, [('binary_file','mask_file')]),
                    
                    (selectfiles,reorient_pw, [('pw_volume','in_file')]),
                    (reorient_pw,reg_pw2anat, [('out_file','in_file')]),
                    (reg_pw2anat,reg_pd2anat, [('out_matrix_file','in_matrix_file')]),
                    (selectfiles,reorient_pd, [('pd_volume','in_file')]),
                    (reorient_pd,reg_pd2anat, [('out_file','in_file')]),
                    (reg_pw2anat,applyMask_pw, [('out_file','in_file')]),
                    (applyMask_pw, quant_cbf_alsop, [('out_file','pw_volume')]),
                    (reg_pd2anat,applyMask_pd, [('out_file','in_file')]),
                    (applyMask_pd,quant_cbf_alsop, [('out_file','pd_volume')]),
                    
                    (quant_cbf_alsop,checkcoreg_cbf, [('cbf_volume','ref')]),
                    (quant_cbf_alsop,reg_cbf2mni,[('cbf_volume','in_file')]),
                    (reg_cbf2mni, cbf_smooth, [('out_file','in_files')]),
                    
                    (checkcoreg_cbf,datasink, [('coreg_file','CBF_anat_coreg')]),
                    (checkcoreg_mni,datasink, [('coreg_file','MNI_anat_coreg')]),
                    (cbf_smooth,datasink, [('smoothed_files','std_cbf')]),
                    (reorient_anat,datasink, [('out_file','preproc_anat')]),
                    (reg_anat2mni,datasink, [('out_file','std_anat')]),
                    (reg_anat2mni,datasink, [('out_matrix_file','reg_mni_xform')]),
                    (binarize,datasink, [('binary_file','brainmask')])                    
                   ])
cbfpreproc.base_dir = wkflow_dir
cbfpreproc.write_graph(graph2use='flat')
cbfpreproc.run('MultiProc', plugin_args={'n_procs': 2})

In [ ]:
def bandpass_filter(in_file, lowpass, highpass, TR):
    import numpy as np
    import nibabel as nb
    from os.path import abspath
    from nipype.interfaces.afni.preprocess import Bandpass
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    out_file = 'func_filtered'
    bp = Bandpass()
    bp.inputs.highpass = highpass
    bp.inputs.lowpass = lowpass
    bp.inputs.in_file = in_file
    bp.inputs.tr = TR
    bp.inputs.out_file = out_file
    bp.inputs.outputtype = 'NIFTI'
    bp.run()
    
    out_file = abspath(out_file + '.nii')
    return(out_file)

def adjust_masks(masks):
    from os.path import abspath
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    from nipype.interfaces.freesurfer.model import Binarize
    #pve0 = csf, pve1 = gm, pve2 = wm
    
    origvols = sorted(masks)
    csf = origvols[0]
    wm = origvols[2]
    vols = []
    
    binary = Binarize()
    binary.inputs.in_file = wm
    binary.inputs.min = 0.5
    binary.inputs.max = 2
    binary.inputs.binary_file = 'WM_seg.nii'
    binary.run()
    wm_new = abspath(binary.inputs.binary_file)
    vols.append(wm_new)
    
    binary2 = Binarize()
    binary2.inputs.in_file = csf
    binary2.erode = 1
    binary2.inputs.min = 0.5
    binary2.inputs.max = 2
    binary2.inputs.binary_file = 'CSF_seg.nii'
    binary2.run()
    csf_new = abspath(binary2.inputs.binary_file)
    vols.append(csf_new)
    
    return(vols)
    
def create_noise_matrix(vols_to_censor,motion_params,comp_noise):
    from numpy import genfromtxt, zeros, column_stack, savetxt
    from os import path
    
    motion = genfromtxt(motion_params, delimiter=None, dtype=None, skip_header=0)
    comp_noise = genfromtxt(comp_noise, delimiter=None, dtype=None, skip_header=1)
    censor_vol_list = genfromtxt(vols_to_censor, delimiter=None, dtype=None, skip_header=0)
    
    try:
        c = censor_vol_list.size
    except:
        c = 0
    
    d=len(comp_noise)

    if c > 1:
        scrubbing = zeros((d,c),dtype=int)
        for t in range(c):
            scrubbing[censor_vol_list[t],t] = 1
        noise_matrix = column_stack((motion,comp_noise,scrubbing))
    elif c == 1:
        scrubbing = zeros((d,c),dtype=int)
        scrubbing[censor_vol_list] = 1
        noise_matrix = column_stack((motion,comp_noise,scrubbing))
    else:
        noise_matrix = column_stack((motion,comp_noise))
    
    noise_file = 'noise_matrix.txt'
    savetxt(noise_file, noise_matrix)
    noise_filepath = path.abspath(noise_file)
    
    return(noise_filepath)

In [ ]:
# Resting state preprocessing nodes

# Trim the initial volumes
trim = Node(Trim(begin_index=vols_to_trim), name='trim')

# Realign each volume to first volume: in_file; out_file, par_file
realign = Node(MCFLIRT(out_file='realigned_func.nii',
                       ref_vol=0), 
               name='realign')

# Slice time correction: in_file, slice_time_corrected_file
slicetime = Node(SliceTimer(time_repetition=rest_TR, 
                            interleaved=interleaved,
                            slice_direction=slice_direction, 
                            out_file='stfunc.nii'), 
                    name='slicetime')

# register the functional volumes to the subject space anat
# inputs: in_file, reference; out_file out_matrix_file
reg_func_to_anat = Node(FLIRT(out_matrix_file='xform.mat'),
                        name='reg_func_to_anat')

apply_reg_to_func = Node(FLIRT(apply_xfm=True, 
                               out_file='warped_func.nii'),  
                         name='apply_reg_to_func')

# Apply binary mask to merged functional scan: in_file, mask_file; out_file
mask_func = Node(ApplyMask(out_file='masked_func.nii'), 
                 name='mask_func')

# Bandpass Filtering all rates are in Hz (1/TR or samples/second)
bandpass = Node(name='bandpass', 
                interface=Function(input_names=['in_file','lowpass','highpass','TR'], 
                                   output_names=['out_file'],
                                   function=bandpass_filter))
bandpass.inputs.lowpass = lowpass_freq
bandpass.inputs.highpass = highpass_freq
bandpass.inputs.TR = rest_TR

# Artifact detection for scrubbing/motion assessment
art = Node(ArtifactDetect(mask_type='file',
                          parameter_source='FSL',
                          norm_threshold=1.0, #mutually exclusive with rotation and translation thresh
                          zintensity_threshold=3,
                          use_differences=[True, False]),
           name='art')

# Segment structural scan
segment = Node(FAST(no_bias=True, 
                    segments=True, 
                    number_classes=3), 
               name='segment')

# Fix the segmentations
fix_confs = Node(name='fix_confs',
                 interface=Function(input_names=['masks'], 
                                    output_names=['vols'],
                                    function=adjust_masks))
# actually run compcor
compcor = Node(CompCor(merge_method='none'), 
               name='compcor')

# Create a denoising mask with compcor + motion
noise_mat = Node(name='noise_mat', interface=Function(input_names=['vols_to_censor','motion_params','comp_noise'],
                                                      output_names=['noise_filepath'], 
                                                      function=create_noise_matrix))

# Denoise the data
denoise = Node(GLM(out_res_name='denoised_residuals.nii', 
                   out_data_name='denoised_func.nii'), 
               name='denoise')

# Register to MNI space
reg_rest2standard = Node(FLIRT(), 
                         name = 'reg_rest2standard')

# Apply the transform to all volumes
rest_applyXform = Node(FLIRT(), 
                       name = 'rest_applyXform')
# Smooth the data
def brightthresh(func):
    import nibabel as nib
    from numpy import median, where
    
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    
    func_nifti1 = nib.load(func)
    func_data = func_nifti1.get_data()
    func_data = func_data.astype(float)
    
    brain_values = where(func_data > 0)
    median_thresh = median(brain_values)
    bright_thresh = 0.75 * median_thresh
    
    return(bright_thresh)
    
brightthresh = Node(Function(input_names=['func'],
                             output_names=['bright_thresh'],
                             function=brightthresh),
                    name='brightthresh')
smooth = Node(SUSAN(fwhm=smoothing_kernel),
              name='smooth')

In [ ]:
def create_coreg_plot(epi,anat):
    from os.path import abspath
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    from nipype.interfaces.nipy.preprocess import Trim
    
    epiVol = 'firstVol.nii'
    trim = Trim()
    trim.inputs.in_file = epi
    trim.inputs.out_file = epiVol
    trim.inputs.end_index = 1
    trim.inputs.begin_index = 0
    trim.run()
    
    coreg_filename='coregistration.png'
    display = plotting.plot_anat(epiVol, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'coregistration to anatomy')
    display.add_edges(anat)
    display.savefig(coreg_filename) 
    display.close()
    coreg_file = abspath(coreg_filename)
    
    return(coreg_file)

def check_mask_coverage(epi,brainmask):
    from os.path import abspath
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from nilearn import plotting
    from nipype.interfaces.nipy.preprocess import Trim
    
    epiVol = 'firstVol.nii'
    trim = Trim()
    trim.inputs.in_file = epi
    trim.inputs.out_file = epiVol
    trim.inputs.end_index = 1
    trim.inputs.begin_index = 0
    trim.run()
    
    maskcheck_filename='maskcheck.png'
    display = plotting.plot_anat(epiVol, display_mode='ortho',
                                 draw_cross=False,
                                 title = 'check brainmask coverage')
    display.add_contours(brainmask,levels=[.5], colors='r')
    display.savefig(maskcheck_filename) 
    display.close()
    checkmask_file = abspath(maskcheck_filename)
    
    return(checkmask_file)
    
make_coreg_img = Node(Function(input_names=['epi','anat'],
                                         output_names=['coreg_file'],
                                         function=create_coreg_plot),
                      name='make_coreg_img')

make_checkmask_img = Node(Function(input_names=['epi','brainmask'],
                                         output_names=['maskcheck_file'],
                                         function=check_mask_coverage),
                          name='make_checkmask_img')

In [ ]:
# workflow
restpreproc = Workflow(name='restpreproc')
restpreproc.connect([(infosource,selectfiles,[('subjid','subjid')]),
                     (infosource,structgrabber,[('subjid','subjid')]),
                     (structgrabber, reg_func_to_anat,[('proc_anat','reference')]),
                     
                     (selectfiles,reorient_func,[('rest','in_file')]),
                     (reorient_func,trim,[('out_file','in_file')]),
                     (trim,realign,[('out_file','in_file')]),
                     (realign, slicetime,[('out_file','in_file')]),
                     (slicetime,reg_func_to_anat,[('slice_time_corrected_file','in_file')]),
                     (slicetime,apply_reg_to_func,[('slice_time_corrected_file','in_file')]),
                     (reg_func_to_anat,apply_reg_to_func,[('out_matrix_file','in_matrix_file')]),
                     (apply_reg_to_func,mask_func,[('out_file','in_file')]),
                     
                     (mask_func,art,[('out_file','realigned_files')]),
                     (realign,art,[('','realignment_parameters')]),
                     (mask_func,compcor,[('out_file','realigned_file')]),
                     (compcor,noise_mat,[('components_file','comp_noise')]),
                     (art,noise_mat,[('outlier_files','vols_to_censor')]),
                     (merge_motion,noise_mat,[('newmotion_params','motion_params')]),
                     (noise_mat,denoise,[('noise_filepath','design')]),
                     (mask_func,denoise,[('out_file','in_file')]),
                     (denoise,bandpass,[('out_data','in_file')]),
                     
                     (mask_func,make_coreg_img,[('out_file','epi')]),
                     (reorient_anat,make_coreg_img,[('out_file','anat')]),
                     (mask_func,make_checkmask_img,[('out_file','epi')]),
                     (binarize_anat,make_checkmask_img,[('binary_file','brainmask')]),
                     
                     (make_coreg_img,datasink,[('coreg_file','coregcheck_image')]),
                     (make_checkmask_img,datasink,[('maskcheck_file','maskcheck_image')]),
                     (mask_func, datasink,[('out_file','orig_merged_func')]),
                     (reorient_anat,datasink,[('out_file','preproc_anat')]),
                     (binarize_anat,datasink,[('binary_file','binarized_anat')]),
                     (noise_mat,datasink,[('noise_filepath','full_noise_mat')]),
                     (art,datasink,[('plot_files','art_plot_files')]),
                     (bandpass,datasink,[('out_file','preproc_func')])        
                    ])
restpreproc.base_dir = workflow_dir
restpreproc.write_graph(graph2use='flat')
restpreproc.run('MultiProc', plugin_args={'n_procs': 2})
